In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys

print(sys.executable)

C:\Users\Erik\PycharmProjects\fallstudie_model_engineering\.venv\Scripts\python.exe


In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split

from src.config import RAW_DATA_PATH, CAT_FEATURES
from src.features import engineer_features, create_categorial_features

from src.models import (
    train_ohc_encoder,
    train_decision_tree,
    train_hgboost,
    tune_hyperparameters,
)
from src.metrics import (
    get_scores,
    plot_confusion_matrix,
    plot_multiple_precision_recall_curves,
    find_best_f1_threshold,
)

In [71]:
raw_data = pd.read_excel(RAW_DATA_PATH, index_col=0)

In [79]:
# Split features and target
data = raw_data.drop_duplicates()
y = data["success"]
X = data.drop(columns=["success"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [80]:
# Engineer Features
X_train = create_categorial_features(data=X_train)
X_test = create_categorial_features(data=X_test)
ohc = train_ohc_encoder(data=X_train[CAT_FEATURES])
X_train = engineer_features(data=X_train, encoder=ohc)
X_test = engineer_features(data=X_test, encoder=ohc)

Train Models

In [11]:
dtm = train_decision_tree(x_train=X_train, y_train=y_train)

In [9]:
hgbm = train_hgboost(x_train=X_train, y_train=y_train)

Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\Erik\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\Erik\PycharmProjects\fallstudie_model_engineering\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 788, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\Erik\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Erik\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1609, in _readerthread
    buffer.append(fh.read())
                  ~~~~~~~^^
  File "C:\Users\Erik\AppData\Local\Programs\Python\Python313\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^

Binning 0.023 GB of training data: 0.035 s
0.001 s 0.003 GB of validation data: 
Fitting gradient boosted rounds:
Fit 46 trees in 3.547 s, (1426 total leaves)
Time spent computing histograms: 0.215s
Time spent finding best splits:  0.027s
Time spent applying splits:      0.032s
Time spent predicting:           0.004s


In [12]:
ohgbm = tune_hyperparameters(x_train=X_train, y_train=y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\Erik\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\Erik\PycharmProjects\fallstudie_model_engineering\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 788, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\Erik\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Erik\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1609, in _readerthread
    buffer.append(fh.read())
                  ~~~~~~~^^
  File "C:\Users\Erik\AppData\Local\Programs\Python\Python313\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^

Binning 0.023 GB of training data: 0.033 s
0.001 s 0.003 GB of validation data: 
Fitting gradient boosted rounds:
Fit 79 trees in 2.949 s, (2449 total leaves)
Time spent computing histograms: 0.360s
Time spent finding best splits:  0.046s
Time spent applying splits:      0.058s
Time spent predicting:           0.007s
Beste Parameter gefunden: {'l2_regularization': np.float64(0.23956189066697242), 'learning_rate': np.float64(0.03897897441824462), 'max_depth': 14, 'max_iter': 317, 'min_samples_leaf': 51}


In [17]:
models_to_evaluate = {
    "decision_tree_model": dtm,
    "hgboost_model": hgbm,
    "optimized_hgboost_model": ohgbm,
    "optimized_hgboost_model_1": ohgbm,
}

In [18]:
scores = []
for name, model in models_to_evaluate.items():
    precision, recall, accuracy, f1, roc_auc, cm = get_scores(
        name=name, model=model, y_true=y_test, x_test=X_test, threshold=0.511
    )
    scores.append(
        {
            "model": name,
            "precision": round(precision, 4),
            "recall": round(recall, 4),
            "accuracy": round(accuracy, 4),
            "f1": round(f1, 4),
            "roc-auc": round(roc_auc, 4),
        }
    )
scores = pd.DataFrame(scores)
print(scores)

                       model  precision  recall  accuracy      f1  roc-auc
0        decision_tree_model     0.5291  0.0524    0.7944  0.0953   0.6565
1              hgboost_model     0.3008  0.7426    0.5898  0.4282   0.6824
2    optimized_hgboost_model     0.3007  0.7368    0.5911  0.4271   0.6837
3  optimized_hgboost_model_1     0.2965  0.7483    0.5808  0.4248   0.6837


In [ ]:
plot_confusion_matrix(x_test=X_test, y_test=y_test, models=models_to_evaluate)

In [ ]:
plot_multiple_precision_recall_curves(
    models=models_to_evaluate, x_test=X_test, y_test=y_test
)

In [ ]:
find_best_f1_threshold(x_test=X_test, y_test=y_test, model=ohgbm)

In [29]:
from src.metrics import feature_importance

feature_importance = get_feature_importance(
    models_to_evaluate=models_to_evaluate, x_test=X_test, y_test=y_test
)

In [34]:
feature_importance.dtm.sort_values(ascending=False).head(20)

Feature
cost_if_failure                                        0.2830
interaction_psp_amount_bin_UK_Card_amount_under_200    0.2019
interaction_psp_3D_secured_UK_Card_0                   0.0894
interaction_psp_card_Simplecard_Master                 0.0818
interaction_psp_3D_secured_Goldcard_0                  0.0681
day                                                    0.0571
timedelta                                              0.0553
is_retry                                               0.0395
hour                                                   0.0376
is_business_hours                                      0.0257
amount                                                 0.0096
day_sin                                                0.0091
is_weekend                                             0.0084
hour_cos                                               0.0066
card_Visa                                              0.0052
interaction_psp_card_UK_Card_Diners                    0.0045


In [34]:
X_test.columns

Index(['tmsp', 'country', 'amount', 'PSP', '3D_secured', 'card'], dtype='object')

In [83]:
from src.predictions import _get_all_predictions, calculate_strategy_kpis

In [81]:
predictions_df = _get_all_predictions(
    model=ohgbm, x_test=X_test, original_data=raw_data, encoder=ohc
)

In [82]:
predictions_df

,original_psp,Moneycard,Goldcard,UK_Card,Simplecard
16866,Simplecard,0.618672,0.720242,0.283769,0.326381
6304,Simplecard,0.412814,0.604694,0.561977,0.490658
43346,Simplecard,0.518547,0.744908,0.233750,0.279756
19952,Simplecard,0.661531,0.708379,0.580122,0.532561
18012,UK_Card,0.426258,0.706814,0.591504,0.377613
...,...,...,...,...,...
32021,Moneycard,0.518727,0.605632,0.262678,0.554731
20378,UK_Card,0.513227,0.651044,0.259483,0.308020
36486,UK_Card,0.697316,0.631357,0.583311,0.363771
13658,UK_Card,0.686494,0.687746,0.574088,0.325204


In [87]:
predictions_df["best_psp"] = "Simplecard"
choices = predictions_df["best_psp"]
calculate_strategy_kpis(choices=choices, y_test=y_test)

(np.float64(0.20683488972779654), 6074.0)

In [88]:
calculate_strategy_kpis(choices=predictions_df["original_psp"], y_test=y_test)

(np.float64(0.20683488972779654), 17927.0)